In [1]:
# Add relevant Jupyter notebook extensions 

In [ ]:
# You can double-check your Python path like this...
import sys  
print(sys.path)

# Simulate closed-loop
After implementing your control functionality, you can simulate the closed-loop with code that looks something like this...

In [ ]:
# Import relevant modules
from dynamic import ClosedLoop, Submarine  #dynamic.py contains these
from control import PDController  # PD controller in control.py
from dynamic import Mission  # To load mission profiles
import matplotlib.pyplot as plt
import numpy as np

sub = Submarine()
# Load the mission data from a CSV file (implement from_csv method in Mission)
mission = Mission.from_csv("D:/Engineering/Year 3/B1/Numerical Algorithms/b1-coding-practical-mt24/data/mission.csv") 

# Set PD controller gains
KP = 0.15  # Proportional gain
KD = 0.6   # Derivative gain

# Function to compute the tracking error (Sum of Squared Errors - SSE)
def compute_error(KP, KD, mission):
    # Instantiate the submarine and PD controller with the given KP and KD
    sub = Submarine()
    controller = PDController(KP, KD)
    closed_loop = ClosedLoop(sub, controller)
    
    # Simulate the closed-loop system
    trajectory = closed_loop.simulate_with_random_disturbances(mission)
    
    # Compute the sum of squared errors (SSE) between reference and actual depth
    # The actual depth is stored within the trajectory, so we need to compare it to the mission reference
    error = np.sum((np.array(mission.reference) - np.array(trajectory.position[:, 1])) ** 2)
    
    return error, trajectory

# Define ranges for KP and KD to test (Grid Search)
KP_values = np.linspace(0.1, 1.0, 10)  # Proportional gains to test
KD_values = np.linspace(0.1, 1.0, 10)  # Derivative gains to test

# Initialize variables to track the best KP, KD, and minimum error
best_KP, best_KD = None, None
min_error = float('inf')
best_trajectory = None

# Perform grid search for optimal KP and KD values
for KP in KP_values:
    for KD in KD_values:
        error, trajectory = compute_error(KP, KD, mission)
        
        # If the error is lower, update the best KP, KD, and min_error
        if error < min_error:
            best_KP, best_KD = KP, KD
            min_error = error
            best_trajectory = trajectory


# Instantiate controller
controller = PDController(best_KP, best_KD)

# Create the closed loop system
closed_loop = ClosedLoop(sub, controller)


# Simulate the closed loop system with random disturbances
trajectory = closed_loop.simulate_with_random_disturbances(mission)

# Plot the completed mission trajectory vs. reference depths
trajectory.plot_completed_mission(mission)